In [4]:
import torch

In [6]:
from transformers import MarianMTModel, MarianTokenizer

In [8]:
en_ROMANCE_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
en_ROMANCE_tokenizer = MarianTokenizer.from_pretrained(en_ROMANCE_model_name)
print(en_ROMANCE_tokenizer.supported_language_codes)

['>>fr<<', '>>es<<', '>>it<<', '>>pt<<', '>>pt_br<<', '>>ro<<', '>>ca<<', '>>gl<<', '>>pt_BR<<', '>>la<<', '>>wa<<', '>>fur<<', '>>oc<<', '>>fr_CA<<', '>>sc<<', '>>es_ES<<', '>>es_MX<<', '>>es_AR<<', '>>es_PR<<', '>>es_UY<<', '>>es_CL<<', '>>es_CO<<', '>>es_CR<<', '>>es_GT<<', '>>es_HN<<', '>>es_NI<<', '>>es_PA<<', '>>es_PE<<', '>>es_VE<<', '>>es_DO<<', '>>es_EC<<', '>>es_SV<<', '>>an<<', '>>pt_PT<<', '>>frp<<', '>>lad<<', '>>vec<<', '>>fr_FR<<', '>>co<<', '>>it_IT<<', '>>lld<<', '>>lij<<', '>>lmo<<', '>>nap<<', '>>rm<<', '>>scn<<', '>>mwl<<']


In [10]:
en_ROMANCE = MarianMTModel.from_pretrained(en_ROMANCE_model_name)

In [ ]:
ROMANCE_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
ROMANCE_en_tokenizer = MarianTokenizer.from_pretrained(ROMANCE_en_model_name)


In [13]:
ROMANCE_en = MarianMTModel.from_pretrained(ROMANCE_en_model_name)

In [110]:
skip = "true"
english = "It was a dark and stormy night"
start = "The"

#english only- spanish behind the scenes
if skip == "true":
    tokenizer = ROMANCE_en_tokenizer
    model = ROMANCE_en

#show spanish
else:
    tokenizer = en_ROMANCE_tokenizer
    model = en_ROMANCE

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(start))
prefix = torch.LongTensor(starting_word)

batch = tokenizer.prepare_translation_batch([">>es<<" + english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id
# pylint: disable=E1101
partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (english_encoded, None)
# pylint: enable=E1101

num_tokens_generated = 0

MAX_LENGTH = 100

prediction_list = []
total = 0
#stop when </s> token generated, or max num tokens exceded (just in case)
while True:#next_token_to_add.item() != 0 and x < MAX_LENGTH:
    model_inputs = model.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
    )
    with torch.no_grad():
        model_outputs = model(**model_inputs)

    next_token_logits = model_outputs[0][0, -1, :]
    past = model_outputs[1]

    #start with user inputted beginning
    if x < len(prefix):
        next_token_to_add = prefix[x]
    else:
        next_token_to_add = next_token_logits[0].topk(100).indices[1].item()
    
    next_token_logprobs = next_token_logits - next_token_logits.logsumexp()
    score = next_token_logprobs[next_token_to_add].item()
    total += score
    
    #     P(this is cool) =       P(this) * P(is | this) * P(cool | this is)
    # log P(this is cool) = log ( P(this) * P(is | this) * P(cool | this is) )
    #                     = log P(this) + log P(is | this) + log P(cool | this is)
    
    # score_A = -5     exp(score_A) <- positive
    # score_B = 2      exp(score_B)
    # 
    #     P(A) = exp(score_A) / ( exp(score_A) + exp(score_B) )
    # log P(A) = score_A - log ( exp(score_A) + exp(score_B) ) )
        
    #append top 10 predictions for each token to list
    decoded_predictions = []
    for tok in next_token_logits[0].topk(10).indices:
        decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

    #list of lists of predictions
    prediction_list.append(decoded_predictions)

    #add new token to tokens so far
    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x += 1
    
    if next_token_to_add.item() == 0 or not (num_tokens_generated < MAX_LEN):
        break

#list of tokens used to display sentence
decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in tokenizer.convert_ids_to_tokens(partial_decode[0])]
decoded_tokens.remove("<pad>")

final = tokenizer.decode(partial_decode[0]).replace("<pad>", '')
print(final)
total/len(decoded_tokens)

AttributeError: 'tuple' object has no attribute 'transpose'

In [76]:
next_token_logits.argmax()

NameError: name 'next_token_logits' is not defined

In [ ]:
next_token_logits[next_token_logits.argmax()]

In [ ]:
import torch

In [85]:
logits = torch.Tensor([-2., 2.]) - 2.0181
logits

tensor([-4.0181, -0.0181])

In [87]:
unnormalized_probs = logits.exp() 
unnormalized_probs

tensor([0.0180, 0.9821])

In [ ]:
probs = unnormalized_probs / unnormalized_probs.sum()

In [90]:
logprobs = probs.log()
logprobs

tensor([-4.0181, -0.0181])

In [91]:
logits - log(sum(exp(logits)))

NameError: name 'log' is not defined

In [92]:
logits.logsumexp(0)

tensor(4.9938e-05)

In [93]:
save[0].topk(11).indices[10]

'' was not found in history, as a file, url, nor in the user namespace.


In [94]:
next_token_logits[0].topk(10).values

NameError: name 'next_token_logits' is not defined

In [95]:
next_token_logits[0].topk(10)

NameError: name 'next_token_logits' is not defined

In [96]:
next_token_logits[0].topk(11).indices

NameError: name 'next_token_logits' is not defined

In [97]:
tokenizer.convert_ids_to_tokens([42])

['▁The']

In [98]:
decoded_tokens

NameError: name 'decoded_tokens' is not defined

In [99]:


tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE
#tokenizer = ROMANCE_en_tokenizer
#model = ROMANCE_en


english = ">>es<<It was a dark and stormy night"
batch = tokenizer.prepare_translation_batch([english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("La"))
#starting_word = tokenizer.encode("A la")

partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (english_encoded, None)

prefix = torch.LongTensor(starting_word)
#prefix = torch.LongTensor(tokenizer.convert_tokens_to_ids("A la".replace(' ', '▁').split('▁')))
#prefix = torch.LongTensor(tokenizer.encode("A la"))
#prefix = torch.LongTensor(tokenizer.encode("A▁la▁"))
next_token_to_add = torch.tensor(1)
x = 0

prediction_list = []

while next_token_to_add.item() != 0 and x < 100:
    print(partial_decode)
    model_inputs = model.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
    )
    with torch.no_grad():
        model_outputs = model(**model_inputs)

    next_token_logits = model_outputs[0][:, -1, :]
    past = model_outputs[1]
    
    if x < len(prefix):
        next_token_to_add = prefix[x]
        print(x, ": ", next_token_to_add)
    else:
        next_token_to_add = next_token_logits[0].argmax()

    decoded_predictions = []
    for tok in next_token_logits[0].topk(5).indices:
        decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))
    
    prediction_list.append(decoded_predictions)
    
    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x += 1
    
decoded_tokens = tokenizer.convert_ids_to_tokens(partial_decode[0])

decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in tokenizer.convert_ids_to_tokens(partial_decode[0])] 

final = tokenizer.decode(partial_decode[0]).split("<pad>")[1]
print(final)
print(decoded_tokens)
print(prediction_list)

batch2 = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<< " + final])
spanish_to_english = ROMANCE_en.generate(**batch2)
new_english = ROMANCE_en_tokenizer.decode(spanish_to_english[0]).split("<pad>")[1]

new_english

tensor([[65000]])


AttributeError: 'tuple' object has no attribute 'transpose'

In [100]:
import random

In [101]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<It was a dark and stormy night"
batch = tokenizer.prepare_translation_batch([english])
english_encoded = model.get_encoder()(**batch)
decoder_start_token = model.config.decoder_start_token_id

starting_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(""))
prefix = torch.LongTensor(starting_word)
prediction_list = []
alternatives = []
for x in range(0, 5):

    past = (english_encoded, None)
   
    partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
    y = 0
    next_token_to_add = torch.tensor(1)
    while next_token_to_add.item() != 0 and y < 100:    
        model_inputs = model.prepare_inputs_for_generation(
        partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=model.config.use_cache
        )
        with torch.no_grad():
            model_outputs = model(**model_inputs)
        next_token_logits = model_outputs[0][:, -1, :]
        past = model_outputs[1]

        if y < len(prefix):
            next_token_to_add = prefix[y]
        else:
            next_token_to_add = next_token_logits[0].topk(4).indices[random.randint(0, 1)]
        decoded_predictions = []
        for tok in next_token_logits[0].topk(5).indices:
            decoded_predictions.append(tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

        prediction_list.append(decoded_predictions)
        partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
        y += 1

    decoded_tokens = tokenizer.convert_ids_to_tokens(partial_decode[0])

    final = tokenizer.decode(partial_decode[0]).split("<pad>")[1]
    print(final)
    alternatives.append(final)
    
alternatives

AttributeError: 'tuple' object has no attribute 'transpose'

In [102]:
decoded_tokens

NameError: name 'decoded_tokens' is not defined

In [103]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<I think maybe this is the wrong store."
batch = tokenizer.prepare_translation_batch([english])
                                            
eng_to_spanish = model.generate(num_beams=50, num_return_sequences=50, **batch)

translations = []
for x in range(0, 50):
    translations.append(tokenizer.decode(eng_to_spanish[x]).split("<pad>")[1])

translations

[' Creo que esta es la tienda equivocada.',
 ' Creo que tal vez esta es la tienda equivocada.',
 ' Creo que quizás esta es la tienda equivocada.',
 ' Creo que puede que esta sea la tienda equivocada.',
 ' Creo que esta tal vez es la tienda equivocada.',
 ' Creo que quizá esta es la tienda equivocada.',
 ' Creo que ésta es la tienda equivocada.',
 ' Creo que es la tienda equivocada.',
 ' Creo que esta puede ser la tienda equivocada.',
 ' Creo que quizá esta sea la tienda equivocada.',
 ' Creo que quizás esta sea la tienda equivocada.',
 ' Creo que tal vez esta sea la tienda equivocada.',
 ' Pienso que esta es la tienda equivocada.',
 ' Creo que a lo mejor esta es la tienda equivocada.',
 ' Creo que puede que esta es la tienda equivocada.',
 ' Creo que tal vez ésta es la tienda equivocada.',
 ' Creo que esta es una tienda equivocada.',
 ' Creo que puede que ésta sea la tienda equivocada.',
 ' Creo que esto es la tienda equivocada.',
 ' Creo que esta tal vez sea la tienda equivocada.',
 '

In [104]:
back_to_english[0]

tensor([65000,    23,    20,   391,   260,   844,    63,    26,     4,  2101,
         5951,     3])

In [105]:
ROMANCE_en_tokenizer.decode(back_to_english[0])

'<pad> - I think maybe this is the wrong store.'

In [106]:
">>en<<" + tokenizer.decode(spanish[0]).replace("<pad> ", '')

'>>en<<( etp▁States doit are des the▁greater leadership.'

In [107]:
back_to_english[0]

tensor([65000,    23,    20,   391,   260,   844,    63,    26,     4,  2101,
         5951,     3])

In [108]:
ROMANCE_en_tokenizer.decode([65000,   4932,    7,    66,   148,   660,  2203,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000])

"<pad>You're my best friend.<pad><pad><pad><pad><pad><pad><pad><pad>"

In [109]:
tokenizer = en_ROMANCE_tokenizer
model = en_ROMANCE

english = ">>es<<You're my soulmate"
batch = tokenizer.prepare_translation_batch([english])

spanish = model.generate(**batch)

batch2 = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<<" + tokenizer.decode(spanish[0])])

back_to_english = ROMANCE_en.generate(num_beams=20, num_return_sequences=20, bad_words_ids=[[1695],[1973],[7927],[55],[23],[367],[51],[12390],[1172],[45351],[39]], **batch2)

translations = []
for x in range(0, 20):
    print(back_to_english[x])
    translations.append(ROMANCE_en_tokenizer.decode(back_to_english[x]).replace("<pad>", ''))
    
translations

tensor([65000,   111,    57,   148,  7112,  9036,     0, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,    57,   148,  7112,  9036,     3,     0, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     0, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,   111,     7,    66,   148,  7112,  9036,     3,   111,     7,
           66,   148,  7112,  9036,     3,     0])
tensor([65000,    31,     7,    66,   148,  7112,  9036,     3,     0, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,    31,    57,   148,  7112,  9036,     0, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000])
tensor([65000,    31,     7,    66,   148,  7112,  9036,     0, 65000, 65000,
        65000, 6500

[' You are my soul mate',
 " You're my soul mate.",
 ' You are my soul mate.',
 " You're my soul mate",
 " You're my soul mate. You're my soul mate.",
 " you're my soul mate.",
 ' you are my soul mate',
 " you're my soul mate",
 " You're my soul mate! You're my soul mate!",
 " You're my soulmate.",
 " you're my soul mate. You're my soul mate.",
 " You know, you're my soul mate.",
 " You're my soul mate. You are my soul mate.",
 " You're my soul mate, you're my soul mate.",
 " You're my soulmate. You're my soulmate.",
 " You're my soul mate!",
 ' you are my soul mate.',
 " You're my soul mate. You're my soulmate.",
 " You're my soul mate. You're my soul mate!",
 ' You are my soulmate.']

In [ ]:
model.generate??

In [112]:
bad_words_ids = [ROMANCE_en_tokenizer.encode(bad_word) for bad_word in ['-', '"']]
bad_words_ids

[[23, 0], [51, 0]]

In [113]:
ROMANCE_en_tokenizer.convert_ids_to_tokens([0])

['</s>']

In [114]:
ROMANCE_en_tokenizer.convert_tokens_to_ids('♪')

47075

In [115]:
english = ">>es<<I think maybe this is the wrong store."
batch = tokenizer.prepare_translation_batch([english])
                                            
spanish = model.generate(**batch)

batch = ROMANCE_en_tokenizer.prepare_translation_batch([">>en<<" + tokenizer.decode(spanish[0]).replace("<pad>", '')])
back_to_english = ROMANCE_en.generate(**batch)
machine_translation = ROMANCE_en_tokenizer.decode(back_to_english[0]).replace("<pad>", '')

spanish_encoded = ROMANCE_en.get_encoder()(**batch)
decoder_start_token = ROMANCE_en.config.decoder_start_token_id
# pylint: disable=E1101
partial_decode = torch.LongTensor([decoder_start_token]).unsqueeze(0)
past = (spanish_encoded, None)
# pylint: enable=E1101
next_token_to_add = torch.tensor(1)
x = 0

prediction_list = []

while next_token_to_add.item() != 0 and x < 100:
    model_inputs = ROMANCE_en.prepare_inputs_for_generation(
    partial_decode, past=past, attention_mask=batch['attention_mask'], use_cache=ROMANCE_en.config.use_cache
    )
    with torch.no_grad():
        model_outputs = ROMANCE_en(**model_inputs)

    next_token_logits = model_outputs[0][:, -1, :]
    past = model_outputs[1]
   
    if x == 3:
        next_token_to_add = next_token_logits[0].topk(4).indices[1]
    else:
        next_token_to_add= next_token_logits[0].argmax()
    decoded_predictions = []
    for tok in next_token_logits[0].topk(10).indices:
        decoded_predictions.append(ROMANCE_en_tokenizer.convert_ids_to_tokens(tok.item()).replace('\u2581', '\u00a0'))

    prediction_list.append(decoded_predictions)

    partial_decode = torch.cat((partial_decode, next_token_to_add.unsqueeze(0).unsqueeze(0)), -1)
    x+= 1

decoded_tokens = [sub.replace('\u2581', '\u00a0') for sub in ROMANCE_en_tokenizer.convert_ids_to_tokens(partial_decode[0])]
decoded_tokens.remove("<pad>")
final = ROMANCE_en_tokenizer.decode(partial_decode[0]).split("<pad>")[1]

final

AttributeError: 'tuple' object has no attribute 'transpose'

In [116]:
machine_translation

' - I think maybe this is the wrong store.'

In [117]:
model.generate??

1;33m
        assert repetition_penalty >= 1.0, "`repetition_penalty` should be >= 1."
        assert input_ids is not None or (
            isinstance(bos_token_id, int) and bos_token_id >= 0
        ), "If input_ids is not defined, `bos_token_id` should be a positive integer."
        assert pad_token_id is None or (
            isinstance(pad_token_id, int) and (pad_token_id >= 0)
        ), "`pad_token_id` should be a positive integer."
        assert (eos_token_id is None) or (
            isinstance(eos_token_id, int) and (eos_token_id >= 0)
        ), "`eos_token_id` should be a positive integer."
        assert length_penalty > 0, "`length_penalty` should be strictly positive."
        assert (
            isinstance(no_repeat_ngram_size, int) and no_repeat_ngram_size >= 0
        ), "`no_repeat_ngram_size` should be a positive integer."
        assert (
            isinstance(num_return_sequences, int) and num_return_sequences > 0
        ), "`num_return_sequences` should be a

In [118]:
next_token_logits[0].topk(3).indices[1]

NameError: name 'next_token_logits' is not defined